In [2]:
import json
import numpy as np


###### Q1.1 ######
def objective_function(X, y, w, lamb):
    """
    Inputs:
    - Xtrain: A 2 dimensional numpy array of data (number of samples x number of features)
    - ytrain: A 1 dimensional numpy array of labels (length = number of samples )
    - w: a numpy array of D elements as a D-dimension weight vector
    - lamb: lambda used in pegasos algorithm

    Return:
    - obj_value: the value of objective function in SVM primal formulation
    """
    # you need to fill in your solution here
    
    # 0.5 * lamb * ||w||^2 + 1/N sum(max(0,1-ywx))  
    X = np.array(X)
    y = np.array(y)

    N = X.shape[0]
    z = 1- np.multiply(y, np.transpose(np.dot(X,w)))
    zmax = z[z>0]
    obj_value = 0.5 * lamb * (np.linalg.norm(w) **2) + np.sum(zmax) / N
    #print(obj_value)

    return obj_value


###### Q1.2 ######
def pegasos_train(Xtrain, ytrain, w, lamb, k, max_iterations):
    """
    Inputs:
    - Xtrain: A list of num_train elements, where each element is a list of D-dimensional features.
    - ytrain: A list of num_train labels
    - w: a numpy array of D elements as a D-dimension vector, which is the weight vector and initialized to be all 0s
    - lamb: lambda used in pegasos algorithm
    - k: mini-batch size
    - max_iterations: the total number of iterations to update parameters

    Returns:
    - learnt w
    - train_obj: a list of the objective function value at each iteration during the training process, length of 500.
    """
    np.random.seed(0)
    Xtrain = np.array(Xtrain)
    ytrain = np.array(ytrain)
    N = Xtrain.shape[0]
    D = Xtrain.shape[1]

    train_obj = []

    for iter in range(1, max_iterations + 1):
        A_t = np.floor(np.random.rand(k) * N).astype(int)  # index of the current mini-batch

        # you need to fill in your solution here
        X_t = Xtrain[A_t, :]
        y_t = ytrain[A_t]
        
        # 4
        #A_tpls = A_t[np.multiply(y_t, np.dot(X_t,w)) < 1]
        A_tpls = A_t[(np.multiply(y_t, np.transpose(np.dot(X_t,w)) )<1).ravel()]
        X_tpls = Xtrain[A_tpls, :]
        y_tpls = ytrain[A_tpls]
        
        # 5
        ita_t = 1/(lamb * iter)
        
        
        # 6
        w_thalf = (1 - (ita_t * lamb)) * w + ita_t * np.sum(
            np.multiply(y_tpls.reshape(y_tpls.shape[0],1),X_tpls),
            axis=0).reshape(D,1)/ k
        
        # 7
        #w = w_thalf * min(1,1/(np.sqrt(lamb) * np.linalg.norm(np.array(w_thalf))))
        w = w_thalf * min(1, 1 / np.sqrt(lamb) / np.linalg.norm(w_thalf))
        
        train_obj.append(objective_function(Xtrain, ytrain, w, lamb))
        print(train_obj[iter-1])
        # print(w[0])
    

    return w, train_obj


###### Q1.3 ######
def pegasos_test(Xtest, ytest, w_l):
    """
    Inputs:
    - Xtest: A list of num_test elements, where each element is a list of D-dimensional features.
    - ytest: A list of num_test labels
    - w_l: a numpy array of D elements as a D-dimension vector, which is the weight vector of SVM classifier and learned by pegasos_train()
 
    Returns:
    - test_acc: testing accuracy.
    """
    # you need to fill in your solution here
    Xtest = np.array(Xtest)
    ytest = np.array(ytest)
    N = Xtest.shape[0]
    ywx = (np.multiply(ytest, np.transpose(np.dot(Xtest, w_l))) > 0).ravel()
    ytru = ytest > 0
    test_acc = sum(ytru == ywx)/N


    return test_acc


"""
NO MODIFICATIONS below this line.
You should only write your code in the above functions.
"""

def data_loader_mnist(dataset):

    with open(dataset, 'r') as f:
            data_set = json.load(f)
    train_set, valid_set, test_set = data_set['train'], data_set['valid'], data_set['test']

    Xtrain = train_set[0]
    ytrain = train_set[1]
    Xvalid = valid_set[0]
    yvalid = valid_set[1]
    Xtest = test_set[0]
    ytest = test_set[1]

    ## below we add 'one' to the feature of each sample, such that we include the bias term into parameter w
    Xtrain = np.hstack((np.ones((len(Xtrain), 1)), np.array(Xtrain))).tolist()
    Xvalid = np.hstack((np.ones((len(Xvalid), 1)), np.array(Xvalid))).tolist()
    Xtest = np.hstack((np.ones((len(Xtest), 1)), np.array(Xtest))).tolist()

    for i, v in enumerate(ytrain):
        if v < 5:
            ytrain[i] = -1.
        else:
            ytrain[i] = 1.
    for i, v in enumerate(ytest):
        if v < 5:
            ytest[i] = -1.
        else:
            ytest[i] = 1.

    return Xtrain, ytrain, Xvalid, yvalid, Xtest, ytest


def pegasos_mnist():

    test_acc = {}
    train_obj = {}

    Xtrain, ytrain, Xvalid, yvalid, Xtest, ytest = data_loader_mnist(dataset = 'mnist_subset.json')

    max_iterations = 500
    k = 100
    for lamb in (0.01, 0.1, 1):
        w = np.zeros((len(Xtrain[0]), 1))
        w_l, train_obj['k=' + str(k) + '_lambda=' + str(lamb)] = pegasos_train(Xtrain, ytrain, w, lamb, k, max_iterations)
        test_acc['k=' + str(k) + '_lambda=' + str(lamb)] = pegasos_test(Xtest, ytest, w_l)

    lamb = 0.1
    for k in (1, 10, 1000):
        w = np.zeros((len(Xtrain[0]), 1))
        w_l, train_obj['k=' + str(k) + '_lambda=' + str(lamb)] = pegasos_train(Xtrain, ytrain, w, lamb, k, max_iterations)
        test_acc['k=' + str(k) + '_lambda=' + str(lamb)] = pegasos_test(Xtest, ytest, w_l)

    return test_acc, train_obj


def main():
    test_acc, train_obj = pegasos_mnist() # results on mnist
    print('mnist test acc \n')
    for key, value in test_acc.items():
        print('%s: test acc = %.4f \n' % (key, value))

    #with open('pegasos.json', 'w') as f_json:
    #    json.dump([test_acc, train_obj], f_json)


#if __name__ == "__main__":
#    main()


## 1.1

In [3]:
Xtrain, ytrain, Xvalid, yvalid, Xtest, ytest = data_loader_mnist(dataset = 'mnist_subset.json')
X = np.array(Xtrain)
y = np.array(ytrain)
N = X.shape[0]
D = X.shape[1]
w = np.ones((D,1))
lamb = 0.5

In [8]:
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score

clf = svm.LinearSVC(C = 0.25,penalty='l2', loss='hinge')
clf.fit(Xtrain, ytrain)
accuracy_score(ytest, clf.predict(Xtest))

C:\Users\Takato\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.842

In [9]:
clf = svm.LinearSVC(C = 0.5,penalty='l2', loss='hinge')
clf.fit(Xtrain, ytrain)
accuracy_score(ytest, clf.predict(Xtest))


C:\Users\Takato\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.846

In [10]:
clf = svm.LinearSVC(C = 0.125,penalty='l2', loss='hinge')
clf.fit(Xtrain, ytrain)
accuracy_score(ytest, clf.predict(Xtest))

C:\Users\Takato\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.844

In [5]:
np.linalg.norm(np.array([3,4]))**2

25.0

In [6]:
np.dot(X,w)

array([[ 41.63671875],
       [101.91015625],
       [130.578125  ],
       ...,
       [190.23828125],
       [ 97.51953125],
       [132.16796875]])

In [7]:
(np.dot(X,w)).shape

(5000, 1)

In [8]:
np.multiply(y, np.dot(X,w))

array([[  41.63671875,   41.63671875,  -41.63671875, ...,  -41.63671875,
          41.63671875,   41.63671875],
       [ 101.91015625,  101.91015625, -101.91015625, ..., -101.91015625,
         101.91015625,  101.91015625],
       [ 130.578125  ,  130.578125  , -130.578125  , ..., -130.578125  ,
         130.578125  ,  130.578125  ],
       ...,
       [ 190.23828125,  190.23828125, -190.23828125, ..., -190.23828125,
         190.23828125,  190.23828125],
       [  97.51953125,   97.51953125,  -97.51953125, ...,  -97.51953125,
          97.51953125,   97.51953125],
       [ 132.16796875,  132.16796875, -132.16796875, ..., -132.16796875,
         132.16796875,  132.16796875]])

In [9]:
(np.multiply(y, np.dot(X,w))).shape

(5000, 5000)

In [10]:
w.shape

(785, 1)

In [11]:
1- np.multiply(y, np.dot(X,w))

array([[ -40.63671875,  -40.63671875,   42.63671875, ...,   42.63671875,
         -40.63671875,  -40.63671875],
       [-100.91015625, -100.91015625,  102.91015625, ...,  102.91015625,
        -100.91015625, -100.91015625],
       [-129.578125  , -129.578125  ,  131.578125  , ...,  131.578125  ,
        -129.578125  , -129.578125  ],
       ...,
       [-189.23828125, -189.23828125,  191.23828125, ...,  191.23828125,
        -189.23828125, -189.23828125],
       [ -96.51953125,  -96.51953125,   98.51953125, ...,   98.51953125,
         -96.51953125,  -96.51953125],
       [-131.16796875, -131.16796875,  133.16796875, ...,  133.16796875,
        -131.16796875, -131.16796875]])

In [12]:
(1- np.multiply(y, np.transpose(np.dot(X,w)))).shape

(1, 5000)

In [13]:
z = 1 - np.multiply(y, np.dot(X,w))
z = z[z>0]

In [14]:
A = np.array([[1,2],[0,1]])
b = np.array([0,1])
np.dot(A,b)

array([2, 1])

## 1.2

In [15]:
k = 1000
max_iterations = 500
w = np.zeros((D,1))
lamb = 0.5

In [16]:
    """
    Inputs:
    - Xtrain: A list of num_train elements, where each element is a list of D-dimensional features.
    - ytrain: A list of num_train labels
    - w: a numpy array of D elements as a D-dimension vector, which is the weight vector and initialized to be all 0s
    - lamb: lambda used in pegasos algorithm
    - k: mini-batch size
    - max_iterations: the total number of iterations to update parameters

    Returns:
    - learnt w
    - train_obj: a list of the objective function value at each iteration during the training process, length of 500.
    """
    np.random.seed(0)
    Xtrain = np.array(Xtrain)
    ytrain = np.array(ytrain)
    N = Xtrain.shape[0]
    D = Xtrain.shape[1]

    train_obj = []

    for iter in range(1, max_iterations + 1):
#    for iter in range(1,2):
        A_t = np.floor(np.random.rand(k) * N).astype(int)  # index of the current mini-batch

        # you need to fill in your solution here
        X_t = Xtrain[A_t, :]
        y_t = ytrain[A_t].astype(int)
        
        # 4
        A_tpls = A_t[(np.multiply(y_t, np.transpose(np.dot(X_t, w)))<1).ravel()]
        X_tpls = Xtrain[A_tpls, :]
        y_tpls = ytrain[A_tpls].astype(int)
        
        # 5
        ita_t = 1/(lamb * iter)
        
        # 6
        w_thalf = (1-ita_t * lamb) * w + (ita_t / k) * np.sum(
            np.multiply(y_tpls.reshape(y_tpls.shape[0], 1) ,X_tpls),
            axis=0).reshape(D,1)
        
        # 7
        #w = w_thalf * min(1,1/(np.sqrt(lamb) * np.linalg.norm(np.array(w_thalf))))
        w = w_thalf * min(1, 1/(np.sqrt(lamb) * np.linalg.norm(w_thalf)))

        train_obj.append(objective_function(X_t, y_t, w, lamb))
        #print(train_obj[iter-1])
        
    #print(train_obj)


    #return w, train_obj

In [17]:
Xtrain[[10, 4, 6],:].shape

(3, 785)

In [18]:
print(X_t.shape)
print(X_tpls.shape)

(1000, 785)
(735, 785)


In [19]:
print(type(train_obj))
print(len(train_obj))

<class 'list'>
500


In [20]:
set(y_tpls.tolist())

{-1, 1}

In [21]:
y_tpls

array([-1, -1,  1,  1,  1,  1, -1, -1, -1,  1,  1, -1,  1,  1, -1, -1, -1,
       -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,
       -1,  1, -1, -1,  1,  1,  1,  1,  1, -1, -1,  1, -1,  1, -1, -1, -1,
        1, -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1,  1,
       -1,  1,  1, -1, -1, -1,  1,  1, -1,  1,  1, -1, -1,  1,  1, -1,  1,
        1, -1,  1, -1,  1,  1, -1, -1, -1,  1,  1,  1, -1,  1, -1,  1, -1,
        1, -1, -1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1, -1, -1,  1,  1,
       -1,  1,  1, -1, -1, -1,  1,  1, -1, -1,  1,  1, -1,  1,  1, -1,  1,
       -1,  1, -1,  1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1,  1,  1,  1,
        1, -1,  1,  1, -1,  1,  1,  1,  1,  1, -1, -1, -1,  1, -1, -1,  1,
        1, -1,  1, -1,  1,  1, -1,  1, -1,  1,  1, -1,  1, -1,  1, -1,  1,
       -1, -1,  1,  1, -1,  1, -1,  1, -1,  1,  1, -1, -1, -1,  1, -1, -1,
       -1,  1, -1,  1,  1

In [22]:
w.shape

(785, 1)

In [23]:
w1 = np.sum(np.multiply(y_tpls.reshape(y_tpls.shape[0],1),X_tpls),axis=0).reshape(D,1)
w1.shape

(785, 1)

In [24]:
print(A_t.shape)
print(A_tpls.shape)
print((np.multiply(y_t, np.dot(X_t,w)) < 1).shape)
print((np.multiply(y_t, np.transpose(np.dot(X_t,w)) )<1).shape)
print((np.multiply(y_t, np.transpose(np.dot(X_t,w)) )<1).ravel())

(1000,)
(735,)
(1000, 1000)
(1, 1000)
[ True  True  True False  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True False  True  True
  True  True False  True  True False  True  True  True  True  True  True
 False False  True False  True False False False  True  True  True  True
  True  True  True  True  True  True  True False False  True False  True
  True False  True False  True  True  True  True  True False False False
  True False  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
  True False False  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True False  True  True  True  True False  True  True  True
  True False 

In [25]:
(np.multiply(y_t, np.dot(X_t,w))).shape

(1000, 1000)

In [26]:
((np.multiply(y_t, np.dot(X_t,w)))<1).shape

(1000, 1000)

In [27]:
np.multiply(y_t, np.dot(X_t,w))

array([[-0.73040117, -0.73040117,  0.73040117, ..., -0.73040117,
        -0.73040117,  0.73040117],
       [-0.67572895, -0.67572895,  0.67572895, ..., -0.67572895,
        -0.67572895,  0.67572895],
       [-0.02825347, -0.02825347,  0.02825347, ..., -0.02825347,
        -0.02825347,  0.02825347],
       ...,
       [ 0.7524917 ,  0.7524917 , -0.7524917 , ...,  0.7524917 ,
         0.7524917 , -0.7524917 ],
       [ 0.3820317 ,  0.3820317 , -0.3820317 , ...,  0.3820317 ,
         0.3820317 , -0.3820317 ],
       [-0.65464567, -0.65464567,  0.65464567, ..., -0.65464567,
        -0.65464567,  0.65464567]])

In [28]:
np.multiply(y_tpls.reshape(y_tpls.shape[0],1), X_tpls)

array([[-1., -0., -0., ..., -0., -0., -0.],
       [-1., -0., -0., ..., -0., -0., -0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [-1., -0., -0., ..., -0., -0., -0.],
       [-1., -0., -0., ..., -0., -0., -0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [29]:
X_t

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [30]:
print(A_t)
print(A_tpls)

[2716 1980  929 3041  557 4739 2351 4039   99 2166 4460 1262 1741 4575
 3074  517 1185 1260 1724  128 2738  989 1059 2283 3626 1569 4644  996
 4424 2149 2707  464  521 4475 2221  555 4033 1752 3710 4295  813 3265
 3948 3239 2290 2161 2890 2070 4099 2188 2098 3445 3040 4042 3542 3950
 4498 1302 3308 4533 4827 1716 1306 2787 1532 2811 3928 3494  828 4260
  625 3560 1750 3570 1266 4139 1085 3161 2202 4076 3314 1527 4193 3263
 3721 3742 2841 4593 4445 4079 2240 3533 1600 1105 3225 1596 3497 2837
  395 1936  924  487 1735 4037   51 1965  866 3855 3172  842 4350 4637
 2286 2810 3982 2627 2822 2783  953 3194 3014 2182 3328 1410 3251 3433
 3867  270 2313 4228 4822  513 4000 1940 1812 3813 4007 3133 3237 4812
 1608 3328 4614 1542  894  945 3658 2718 3057 4633 2246 1123 2204 3691
 3456 4976  199 4362 2347 2400 1349 4180 4025 1798  923 3461 3845   12
 2553 4392  969 2796 2138 4681  278 4098 3810 3190  421  813 2784 1982
 1363 1287  349 3710 3310 2595 3855 2869 2480 1689 1982 2981 2542 1710
  919 

In [31]:
Xtrain[A_t].shape[0]

1000

In [32]:
A_tpls

array([2716, 1980,  929,  557, 4739, 2351, 4039,   99, 2166, 4460, 1262,
       4575, 3074,  517, 1185, 1260, 1724,  128, 2738, 1059, 2283, 3626,
       1569,  996, 4424, 2707,  464,  521, 4475, 2221,  555, 3710,  813,
       2290, 2161, 2890, 2070, 4099, 2188, 2098, 3445, 3040, 4042, 3542,
       1302, 4533, 4827, 1306, 1532, 2811, 3928, 3494,  828, 1750, 1266,
       4139, 1085, 3161, 2202, 3314, 1527, 4193, 3263, 3721, 3742, 2841,
       4593, 4445, 4079, 2240, 3533, 1600, 1105, 3225, 1596, 3497,  395,
       1936,  924,  487, 1735, 4037,   51, 1965,  866, 3855, 3172, 4637,
       2286, 2810, 3982, 2627, 2783,  953, 3194, 3014, 2182, 1410, 3251,
       3433, 3867,  270, 4228, 4822,  513, 4000, 1940, 1812, 3813, 4007,
       3133, 3237, 1608, 4614, 1542,  894,  945, 3658, 3057, 4633, 2246,
       1123, 3691, 3456, 4976,  199, 2347, 1349, 4180, 4025, 1798,  923,
       3461, 3845,   12, 4392,  969, 2796, 2138,  278, 4098, 3810,  421,
        813, 2784, 1982, 1363, 1287, 3710, 3310, 25

In [33]:
ita_t

0.004

In [34]:
A = np.array([[1,2],[1,1],[0,1]])
b = np.array([1,-1,1])
np.multiply(b.reshape(b.shape[0],1),A)

array([[ 1,  2],
       [-1, -1],
       [ 0,  1]])

In [35]:
min(1,2)

1

In [36]:
w_thalf.shape

(785, 1)

In [37]:
c=[]

In [38]:
c.append(1)

In [39]:
c

[1]

In [40]:
iter

500

In [41]:
objective_function(X_t, y_t, w, lamb)

0.6257762095295042

In [42]:
objective_function(X_t, y_t, np.zeros(X_t.shape[1]), lamb)

1.0

In [43]:
np.dot(w,w.reshape(w.shape[0],1))

ValueError: shapes (785,1) and (785,1) not aligned: 1 (dim 1) != 785 (dim 0)

In [ ]:
np.linalg.norm(w) **2

In [ ]:
y_tpls[0:4]

In [ ]:
X_tpls[0:4, :]

In [ ]:
np.multiply(y_tpls.reshape(y_tpls.shape[0],1),X_tpls)[0:4, :]

In [ ]:
 np.sum(np.multiply(y_tpls.reshape(y_tpls.shape[0],1),X_tpls),axis=0).shape

In [ ]:
type(np.sum(np.multiply(y_tpls.reshape(y_tpls.shape[0],1),X_tpls),axis=0))

## 1.3

In [ ]:
    max_iterations = 500
    k = 100
    lamb = 0.1
    w = np.zeros((len(Xtrain[0]), 1))
    w_l, train_obj= pegasos_train(Xtrain, ytrain, w, lamb, k, max_iterations)

In [ ]:
    """
    Inputs:
    - Xtest: A list of num_test elements, where each element is a list of D-dimensional features.
    - ytest: A list of num_test labels
    - w_l: a numpy array of D elements as a D-dimension vector, which is the weight vector of SVM classifier and learned by pegasos_train()
 
    Returns:
    - test_acc: testing accuracy.
    """
    # you need to fill in your solution here
    Xtest = np.array(Xtest)
    ytest = np.array(ytest)
    N = Xtest.shape[0]
    #ywx = (np.multiply(ytest, np.transpose(np.dot(Xtest,w_l))) > 0).ravel()
    ywx = np.sign((np.multiply(ytest, np.transpose(np.dot(Xtest,w_l)))).ravel())
    #ytru = ytest > 0
    #test_acc = sum(ytru == ywx)/N
    test_acc = sum(ytest == ywx)/N


    #return test_acc

In [ ]:
w.shape

In [ ]:
test_acc

In [ ]:
len(train_obj)

In [ ]:
np.multiply(ytest, np.transpose(np.dot(Xtest,w_l))).shape

In [ ]:
ytest.shape

In [ ]:
ywx = (np.multiply(ytest, np.transpose(np.dot(Xtest,w_l))) > 0).ravel()
ytru = ytest > 0
sum(ytru * 1 == ywx * 1)

In [ ]:
ytru*1

In [ ]:
yytr=[True, True, False, False]
yyte=[True, False, False, False]
type(yyte)
#sum(yytr == yyte)

In [ ]:
a= np.array([1,2]) > 1
b = np.array([[-1],[2]])>0
a==b

In [ ]:
a==np.transpose(b)

In [ ]:
li = [1,2,3]

In [ ]:
ywx

In [ ]:
ytru

In [ ]:
w_l.shape

In [ ]:
Xtrain.shape

In [ ]:
g = np.array([[1,1],[1,0],[0,1],[-1,-1]])
wg = np.array([[2],[4]])
np.dot(g,wg)